# <center><b> Python Programming in Energy Science I</b></center>

## <center> Assignment A2 </center>

### <center>[Group 4]</center>
<center> Juan Manuel Boullosa Novo </center>

-------------------------------

### E1. Data analysis

In [169]:
class WindAnalysis:
    def __init__(self, filepath):
        self.datestamps, self.wind_speed_vectors = self._read_csv(filepath)
        self.magnitudes = [self._vector_magnitude(vector) for vector in self.wind_speed_vectors]
        self.mean, self.mean_mag, self.std = self._mean_vector()
        self.is_positive, self.is_over_25 = self._check_wind_speeds()
        self.min, self.max = self._find_min_max()
    '''
    PRIVATE methods called by the constructor or other class methods:
    '''
    def _read_csv(self, filepath):
        with open(filepath, 'r') as file:
            next(file)  # skip header
            data = [line.strip().split(',') for line in file]
        datestamps = [row[0] for row in data]
        wind_speed_vectors = [[float(row[1]), float(row[2])] for row in data]
        return datestamps, wind_speed_vectors
    
    def _vector_magnitude(self, vector):
        return (vector[0] ** 2 + vector[1] ** 2) ** 0.5
    
    def _mean_vector(self):
        vector_length = len(self.wind_speed_vectors)
        sum_u100, sum_v100 = sum(float(row[0]) for row in self.wind_speed_vectors), sum(float(row[1]) for row in self.wind_speed_vectors)
        avg_vector = [sum_u100 / vector_length, sum_v100 / vector_length]
        avg_vector_magnitude = self._vector_magnitude(avg_vector)
        std_dev = self._standard_deviation(avg_vector)
        return avg_vector, avg_vector_magnitude, std_dev
        
    def _check_wind_speeds(self):
        return all(mag > 0 for mag in self.magnitudes), any(mag > 25 for mag in self.magnitudes)

    def _find_min_max(self):
        return min(self.magnitudes), max(self.magnitudes)
    '''
    PRIVATE HELPER methods that are not called by the user:
    '''    
    def _standard_deviation(self, mean_vector):
        # Handled by mean_vector() method
        squared_differences = [(float(vector[0]) - mean_vector[0]) ** 2 + (float(vector[1]) - mean_vector[1]) ** 2 for vector in self.wind_speed_vectors]
        variance = sum(squared_differences) / len(squared_differences)
        return variance ** 0.5
    
    def _time_difference(self, date1, date2):
        # Calculate the difference in seconds between two dates
        year1, month1, day1, hour1, minute1, second1 = map(int, date1.replace('-', ' ').replace(':', ' ').split())
        year2, month2, day2, hour2, minute2, second2 = map(int, date2.replace('-', ' ').replace(':', ' ').split())

        time1 = second1 + minute1*60 + hour1*3600 + day1*86400 + month1*2592000 + year1*31104000
        time2 = second2 + minute2*60 + hour2*3600 + day2*86400 + month2*2592000 + year2*31104000
        return abs(time2 - time1)
    
    def _s_to_hms(self, seconds):
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return hours, minutes, seconds
    
    def _float_range(self, start, stop, step):
        # Custom generator function to create a range of floats
        while start < stop:
            yield start
            start += step
    '''
    PUBLIC methods that can be called by the user:
    '''
    def bin_counter(self, bin_size=1):
        # Sorts wind speeds into bins (dict) of size bin_size from min to max
        bins = dict()
        for bin in self._float_range(self.min, self.max, bin_size):
            bins[bin] = sum(1 for magnitude in self.magnitudes if bin <= magnitude < bin+1)
        return bins

    def longest_period_greater_than(self, threshold=10):
        longest_period, current_period = 0, 0
        previous_date = None
        
        for i in range(len(self.wind_speed_vectors)):
            magnitude = self._vector_magnitude(self.wind_speed_vectors[i])
            date = self.datestamps[i]
            
            if magnitude > threshold:
                if previous_date is not None:
                    current_period += self._time_difference(previous_date, date)
                previous_date = date
            else:
                longest_period = max(longest_period, current_period)
                current_period = 0

        longest_period = self._s_to_hms(max(longest_period, current_period))
        return longest_period
'''
MAIN PROGRAM:
'''
# Call instance of class with csv file path as constructor argument:
wind_analysed = WindAnalysis('winddata.csv')
# Call optional methods:
bins = wind_analysed.bin_counter()
longest_period10 = wind_analysed.longest_period_greater_than()
longest_period15 = wind_analysed.longest_period_greater_than(15)
longest_period25 = wind_analysed.longest_period_greater_than(25)

# Print results:
print(f'Average wind speed vector: [{wind_analysed.mean[0]:.4f}, {wind_analysed.mean[1]:.4f}]')
print(f'Mean windspeed: {wind_analysed.mean_mag:.4f} m/s')
print(f'Standard deviation: {wind_analysed.std:.4f}')
print(f'All wind speeds are positive: {wind_analysed.is_positive}')
print(f'Any wind speed is over 25 m/s: {wind_analysed.is_over_25}')
print(f'Minimum and maximum wind speeds: [{wind_analysed.min:.4f} m/s, {wind_analysed.max:.4f} m/s]')
print(f'\nCount of wind speeds in bins of size 1 m/s:\n')
for key, value in bins.items():
    print(f'{format(key, ".4f"):>7} m/s -> {value:>3} counts')
print(f'\nLongest period of wind speed greater than 10 m/s:\n[{longest_period10[0]} hours, {longest_period10[1]} minutes, {longest_period10[2]} seconds]')
print(f'\nLongest period of wind speed greater than 15 m/s:\n[{longest_period15[0]} hours, {longest_period15[1]} minutes, {longest_period15[2]} seconds]')
print(f'\nLongest period of wind speed greater than 25 m/s:\n[{longest_period25[0]} hours, {longest_period25[1]} minutes, {longest_period25[2]} seconds]')

Average wind speed vector: [1.9637, 0.4262]
Mean windspeed: 2.0095 m/s
Standard deviation: 6.3649
All wind speeds are positive: True
Any wind speed is over 25 m/s: False
Minimum and maximum wind speeds: [0.0621 m/s, 23.1355 m/s]

Count of wind speeds in bins of size 1 m/s:

 0.0621 m/s -> 134 counts
 1.0621 m/s -> 408 counts
 2.0621 m/s -> 628 counts
 3.0621 m/s -> 619 counts
 4.0621 m/s -> 616 counts
 5.0621 m/s -> 564 counts
 6.0621 m/s -> 459 counts
 7.0621 m/s -> 399 counts
 8.0621 m/s -> 425 counts
 9.0621 m/s -> 318 counts
10.0621 m/s -> 174 counts
11.0621 m/s -> 131 counts
12.0621 m/s ->  86 counts
13.0621 m/s ->  46 counts
14.0621 m/s ->  34 counts
15.0621 m/s ->  16 counts
16.0621 m/s ->   9 counts
17.0621 m/s ->   9 counts
18.0621 m/s ->   4 counts
19.0621 m/s ->   6 counts
20.0621 m/s ->   1 counts
21.0621 m/s ->   1 counts
22.0621 m/s ->   0 counts
23.0621 m/s ->   1 counts

Longest period of wind speed greater than 10 m/s:
[737 hours, 0 minutes, 0 seconds]

Longest period 

### E2. Database

In [6]:
table = {
    "Julia": {"Age": 3, "Color": "Green"},
    "Jim": {"Age": 32, "Color": "Red"},
    "Marco": {"Age": 16, "Color": "Green"},
    "Denise": {"Age": 23, "Color": "Blue"},
    "Paula": {"Age": 28, "Color": "Red"},
    "Louis": {"Age": 42, "Color": "Yellow"}
}

def get_person_info(table, name):
    if name not in table:
        raise KeyError(f"No person named {name} in the database.")
    return table[name]['Age'], table[name]['Color']

def get_people_by_color(table, color):
    people = [name for name, person in table.items() if person['Color'] == color]
    if not people:
        raise ValueError(f"No person associated with color {color} in the database.")
    return people

def get_people_by_age_range(table, min_age, max_age):
    people = [name for name, person in table.items() if min_age <= person['Age'] <= max_age]
    if not people:
        return (f"No person between {min_age} and {max_age} years old in the database.")
    return people

print("People:",*table)

name = input("Enter name: ")
age, color = get_person_info(table, name)
print(f"\n{name} is {age} years old and likes color {color}")

color = input("Enter color: ")
people = get_people_by_color(table, color)
print(f"\nPeople who like color {color} are:")
print(', '.join(people))

min_age, max_age = map(int, input("Enter min and max age: ").split())
people = get_people_by_age_range(table, min_age, max_age)
print(f"\nPeople between {min_age} and {max_age} years old are:")
print(*people)

People: Julia Jim Marco Denise Paula Louis

Julia is 3 years old and likes color Green

People who like color Red are:
Jim, Paula

People between 20 and 50 years old are:
Jim Denise Paula Louis


### E3. Poetry generator

In [103]:
from numpy.random import randint

subjects = ["Dog", "Cat", "Bird", "Fish", "Elephant", "Lion", "Tiger", "Bear"]
verbs = ["runs", "jumps", "flies", "swims", "eats", "sleeps", "plays", "sings"]
objects = ["ball", "fish", "mouse", "tree", "leaf", "rock", "stick", "flower"]
comments = ["quickly.", "slowly,", "happily!", "sadly?", "quietly.", "loudly,", "carefully!", "recklessly?"]

sentence_data = {
    "subject": subjects,
    "verb": verbs,
    "object": objects,
    "comment": comments
}

def random_word_generator(list):
    yield list[randint(0, len(list))]

def random_sentence(data):
    subject = random_word_generator(data["subject"])
    verb = random_word_generator(data["verb"])
    object = random_word_generator(data["object"])
    comment = random_word_generator(data["comment"])
    return f"{next(subject)} {next(verb)} {next(object)} {next(comment)}"

for i in range(3):
    print(random_sentence(sentence_data))

Lion sings ball happily!
Dog plays rock quickly.
Elephant eats fish recklessly?


### E4. Classic phone book

In [19]:
class PhoneBook:
    def __init__(self):
        self.phone_book = {}
        
    def add_contact(self, name, number):
        if name in self.phone_book:
            raise ValueError(f"{name} already exists in the phone book. Please enter a new name.")
        self.phone_book[name] = number
        
    def list_contacts(self):
        if not self.phone_book:
            raise ValueError("The phone book is empty.")
        return "\n".join(f"{name}" for name in self.phone_book.keys())
            
    def get_number(self, name):
        if name not in self.phone_book:
            raise ValueError(f"{name} does not exist in the phone book. Please enter another name.")
        return self.phone_book[name]

phone_book = PhoneBook()
commands = {'add': phone_book.add_contact, 'list': phone_book.list_contacts, 'get': phone_book.get_number}

while True:
        command = input("Enter command (add, list, get, quit): ")
        
        if command == 'quit':
            break
        
        elif command == 'add':
            name = input("Enter name: ")
            number = input("Enter number: ")
            commands[command](name, number)
            
        elif command == 'list':
            print(commands[command]())
            
        elif command == 'get':
            name = input("Enter name: ")
            number = commands[command](name)
            print(f"{name}: {number}")
            
        else:
            print("Invalid command.")

Invalid command.
Invalid command.
Juan
Pepe
Invalid command.
Pepe: 5


### E5. String analysis

### E6. Fibonacci sequence